In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [2]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_zhishi_sheke=pd.DataFrame(columns=columns)
channel_zhishi_sheke

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [3]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [4]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [10]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[7]/div/a/span').click()  #选择“知识”频道

In [5]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[7]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'知识'

In [6]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[3]/a').click()  #选择“社科人文”子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[3]/a').text
sub_channel

'社科人文'

In [8]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [9]:
print(datetime.datetime.now())

for month in range(1,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, 700)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_zhishi_sheke.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 

2020-12-21 07:49:23.520188


<ipython-input-9-e13bedf961e5>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-9-e13bedf961e5>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-9-e13bedf961e5>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1UJ41157Lw视频属性的抓取
BV号为1UJ41157Lw的视频属性抓取结束


<ipython-input-9-e13bedf961e5>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为197411v7PA视频属性的抓取
BV号为197411v7PA的视频属性抓取结束
开始BV号为17J411p7iR视频属性的抓取
BV号为17J411p7iR的视频属性抓取结束
开始BV号为1i7411B7kW视频属性的抓取
BV号为1i7411B7kW的视频属性抓取结束
开始BV号为1Q7411e7uq视频属性的抓取
BV号为1Q7411e7uq的视频属性抓取结束
开始BV号为1AJ411V7qg视频属性的抓取
BV号为1AJ411V7qg的视频属性抓取结束
开始BV号为1k7411Y7Ss视频属性的抓取
BV号为1k7411Y7Ss的视频属性抓取结束
开始BV号为1YJ411H7hY视频属性的抓取
BV号为1YJ411H7hY的视频属性抓取结束
开始BV号为1w7411k7LN视频属性的抓取
BV号为1w7411k7LN的视频属性抓取结束
开始BV号为1DJ411H7cb视频属性的抓取
BV号为1DJ411H7cb的视频属性抓取结束
开始BV号为1c7411B72c视频属性的抓取
BV号为1c7411B72c的视频属性抓取结束
开始BV号为1dJ41157Je视频属性的抓取
BV号为1dJ41157Je的视频属性抓取结束
开始BV号为14J41157fS视频属性的抓取
BV号为14J41157fS的视频属性抓取结束
开始BV号为1jJ411E7EC视频属性的抓取
BV号为1jJ411E7EC的视频属性抓取结束
开始BV号为1BJ411j7vM视频属性的抓取
BV号为1BJ411j7vM的视频属性抓取结束
开始BV号为15J411L7vd视频属性的抓取
BV号为15J411L7vd的视频属性抓取结束
开始BV号为1E741167P7视频属性的抓取
BV号为1E741167P7的视频属性抓取结束
开始BV号为1PJ411j7w7视频属性的抓取
BV号为1PJ411j7w7的视频属性抓取结束
开始BV号为1D7411B7PA视频属性的抓取
BV号为1D7411B7PA的视频属性抓取结束
开始BV号为1CJ411J7HX视频属性的抓取
BV号为1CJ411J7HX的视频属性抓取结束
开始BV号为1n7411q7Ho视频属性的抓取
BV号为1n7411q7Ho的视频属性抓取结束
开始BV号为117411k7sT视频属性的抓取
BV号为117411k7sT的视

...

开始BV号为1SC4y1h7SA视频属性的抓取
BV号为1SC4y1h7SA的视频属性抓取结束
开始BV号为1zf4y1174K视频属性的抓取
BV号为1zf4y1174K的视频属性抓取结束
开始BV号为1ZC4y1h7Cn视频属性的抓取
BV号为1ZC4y1h7Cn的视频属性抓取结束
开始BV号为1wv411B71E视频属性的抓取
BV号为1wv411B71E的视频属性抓取结束
开始BV号为1pV411r7xY视频属性的抓取
BV号为1pV411r7xY的视频属性抓取结束
开始BV号为1VK411H7hc视频属性的抓取
BV号为1VK411H7hc的视频属性抓取结束
开始BV号为18C4y1h7qb视频属性的抓取
BV号为18C4y1h7qb的视频属性抓取结束
开始BV号为1c54y1B77N视频属性的抓取
BV号为1c54y1B77N的视频属性抓取结束
开始BV号为1az4y1D7rQ视频属性的抓取
BV号为1az4y1D7rQ的视频属性抓取结束
开始BV号为1kK411J7LL视频属性的抓取
BV号为1kK411J7LL的视频属性抓取结束
开始BV号为17K4y1s78s视频属性的抓取
BV号为17K4y1s78s的视频属性抓取结束
开始BV号为1vC4y1b7Mg视频属性的抓取
BV号为1vC4y1b7Mg的视频属性抓取结束
开始BV号为1wp4y1S7qt视频属性的抓取
BV号为1wp4y1S7qt的视频属性抓取结束
开始BV号为1nz4y1D7x4视频属性的抓取
BV号为1nz4y1D7x4的视频属性抓取结束
开始BV号为1sp4y1q7tF视频属性的抓取
BV号为1sp4y1q7tF的视频属性抓取结束
开始BV号为1zC4y1h7k8视频属性的抓取
BV号为1zC4y1h7k8的视频属性抓取结束
开始BV号为1Zp4y1U7wU视频属性的抓取
BV号为1Zp4y1U7wU的视频属性抓取结束
开始BV号为1HD4y1U7wU视频属性的抓取
BV号为1HD4y1U7wU的视频属性抓取结束
开始BV号为1uK4y1x7B2视频属性的抓取
BV号为1uK4y1x7B2的视频属性抓取结束
开始BV号为1zi4y1G7qM视频属性的抓取
BV号为1zi4y1G7qM的视频属性抓取结束
开始BV号为1UT4y177o6视频属性的抓取
BV号为1UT4y177o6的视

<ipython-input-9-e13bedf961e5>:187: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1Tz411v7Gw视频属性的抓取
BV号为1Tz411v7Gw的视频属性抓取结束
开始BV号为1ui4y137d3视频属性的抓取
BV号为1ui4y137d3的视频属性抓取结束
开始BV号为1G5411e7HN视频属性的抓取
BV号为1G5411e7HN的视频属性抓取结束
开始BV号为1vk4y1m7g2视频属性的抓取
BV号为1vk4y1m7g2的视频属性抓取结束
开始BV号为1dv411i771视频属性的抓取
BV号为1dv411i771的视频属性抓取结束
开始BV号为11a4y1E79f视频属性的抓取
BV号为11a4y1E79f的视频属性抓取结束
开始BV号为175411Y71s视频属性的抓取
BV号为175411Y71s的视频属性抓取结束
开始BV号为1Ff4y197tD视频属性的抓取
BV号为1Ff4y197tD的视频属性抓取结束
开始BV号为1zp4y1i71y视频属性的抓取
BV号为1zp4y1i71y的视频属性抓取结束
开始BV号为1fT4y1L7hr视频属性的抓取
BV号为1fT4y1L7hr的视频属性抓取结束
开始BV号为1m5411h7SH视频属性的抓取
BV号为1m5411h7SH的视频属性抓取结束
开始BV号为1EV411r7U4视频属性的抓取
BV号为1EV411r7U4的视频属性抓取结束
开始BV号为1BV41167wb视频属性的抓取
BV号为1BV41167wb的视频属性抓取结束
开始BV号为1Ba4y1E7qX视频属性的抓取
BV号为1Ba4y1E7qX的视频属性抓取结束
开始BV号为1za4y1a7pV视频属性的抓取
BV号为1za4y1a7pV的视频属性抓取结束
开始BV号为1cC4y1Y74g视频属性的抓取
BV号为1cC4y1Y74g的视频属性抓取结束
开始BV号为1Kf4y1d7dS视频属性的抓取
BV号为1Kf4y1d7dS的视频属性抓取结束
开始BV号为1La4y1e74p视频属性的抓取
BV号为1La4y1e74p的视频属性抓取结束
开始BV号为1up4y1S7df视频属性的抓取
BV号为1up4y1S7df的视频属性抓取结束
开始BV号为15T4y1E7E6视频属性的抓取
BV号为15T4y1E7E6的视频属性抓取结束
开始BV号为1Fk4y167Eg视频属性的抓取
BV号为1Fk4y167Eg的视

开始BV号为1Ev411C7sD视频属性的抓取
BV号为1Ev411C7sD的视频属性抓取结束
开始BV号为1m54y1y7hj视频属性的抓取
BV号为1m54y1y7hj的视频属性抓取结束
开始BV号为16T4y1w7dY视频属性的抓取
BV号为16T4y1w7dY的视频属性抓取结束
开始BV号为1Y64y1F7cR视频属性的抓取
BV号为1Y64y1F7cR的视频属性抓取结束
开始BV号为1Z54y1e7nz视频属性的抓取
开始BV号为1Z54y1e7nz视频属性的抓取
BV号为1Z54y1e7nz的视频属性抓取结束
开始BV号为17a4y1j7Ch视频属性的抓取
BV号为17a4y1j7Ch的视频属性抓取结束
开始BV号为1y5411b7he视频属性的抓取
BV号为1y5411b7he的视频属性抓取结束
开始BV号为1dD4y1R7km视频属性的抓取
BV号为1dD4y1R7km的视频属性抓取结束
开始BV号为15Z4y1T7cA视频属性的抓取
BV号为15Z4y1T7cA的视频属性抓取结束
开始BV号为1Zf4y1S7wv视频属性的抓取
BV号为1Zf4y1S7wv的视频属性抓取结束
开始BV号为15h41197p5视频属性的抓取
BV号为15h41197p5的视频属性抓取结束
开始BV号为1Ra4y177Zf视频属性的抓取
BV号为1Ra4y177Zf的视频属性抓取结束
开始BV号为1ZK4y1h7fS视频属性的抓取
BV号为1ZK4y1h7fS的视频属性抓取结束
开始BV号为1fa4y157cN视频属性的抓取
BV号为1fa4y157cN的视频属性抓取结束
开始BV号为1ya4y1j7Mo视频属性的抓取
BV号为1ya4y1j7Mo的视频属性抓取结束
开始BV号为1454y1m7bF视频属性的抓取
BV号为1454y1m7bF的视频属性抓取结束
开始BV号为1HV411m7Sy视频属性的抓取
BV号为1HV411m7Sy的视频属性抓取结束
开始BV号为1kp4y1e7VH视频属性的抓取
BV号为1kp4y1e7VH的视频属性抓取结束
开始BV号为1qK411P7yZ视频属性的抓取
BV号为1qK411P7yZ的视频属性抓取结束
开始BV号为1Xt4y1S7sH视频属性的抓取
BV号为1Xt4y1S7sH的视频属性抓取结束
开始BV号为1Kt4y1q7RR

BV号为1Pt4y1e7zw的视频属性抓取结束
开始BV号为1Lv411y7vg视频属性的抓取
BV号为1Lv411y7vg的视频属性抓取结束
开始BV号为1r541157xa视频属性的抓取
BV号为1r541157xa的视频属性抓取结束
开始BV号为15v411k7YJ视频属性的抓取
BV号为15v411k7YJ的视频属性抓取结束
开始BV号为1dt4y1Y78L视频属性的抓取
BV号为1dt4y1Y78L的视频属性抓取结束
开始BV号为1Fk4y1C7Rb视频属性的抓取
BV号为1Fk4y1C7Rb的视频属性抓取结束
开始BV号为1iD4y197CS视频属性的抓取
BV号为1iD4y197CS的视频属性抓取结束
2020-12-21 20:57:02.313446


In [29]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-29-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [10]:
channel_zhishi_sheke.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
1月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
2月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
3月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
4月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
5月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
6月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
7月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
8月,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98


In [11]:
channel_zhishi_sheke.to_json('channel_zhishi_sheke.json')

In [44]:
channel_yinyue_dianyin=channel_yinyue_dianyin.iloc[:793]

In [32]:
channel_yinyue_dianyin

,时间,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
1F7411k7ED,1月,U0__,无,4971,90,草，我做了首歌,音乐,电音,活动作品,自制,"[编曲, 钢琴, 电音, 演奏, 音乐, 电音, bilibili新星计划, 草, FLST...",00:34,149.6万,10.5万,1.1万,3.4万,6216,2055,1.1万,否
1W7411Y7YV,1月,迅雷Thunder,无,1.1万,129,变成8bit了啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊,音乐,电音,活动作品和合作视频,自制,"[电音, 音乐, 电音, 薔薇より美しい, 8-bit, 全民音乐UP主, 你比玫瑰更美, ...",01:42,48.2万,3.2万,1.0万,1.6万,2296,772,1179,否
1P7411B7uQ,1月,IR_Mokou,个人认证,9319,119,屑 の 角 色 曲 [Money],音乐,电音,活动作品,自制,"[电音, 音乐, 电音, 屑, 神楽Mea, bilibili新星计划, fl]",01:23,44.5万,2.2万,4684,9744,1327,309,1510,否
1W7411a7Vh,1月,KiraraMagic,个人认证,7.8万,56,JOJO的奇妙冒险DIO送葬曲【电音Remix】,音乐,电音,普通视频,自制,"[JOJO的奇妙冒险, 电音, 音乐, 电音, DIO, VOODOO KINGDOM, K...",03:20,43.6万,2.5万,1.3万,2.0万,1937,819,1545,否
1Q7411r7Bg,1月,皮卡丘本皮,个人认证,16.4万,67,你可听过这么暴躁的《好运来》？,音乐,电音,普通视频,自制,"[电音, 音乐, 电音, Brostep, Trap, Hybrid Trap, 春节歌曲,...",01:05,39.8万,2.5万,8283,1.6万,4252,736,1558,否
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1a64y1c77o,8月,Big_Flowey,无,7049,26,[undertale AU] 审判曲大串烧（不全）,音乐,电音,普通视频,自制,"[电音, 音乐, 电音, AU, 串烧, 审判曲, 狂妄之人, UNDERTALEAU]",12:06,7694,512,101,320,7,42,852,否
1iD4y127xm,8月,奶茶喵w,无,3363,28,Underverse - nightmare BlackApple 地域之诗 nm终于有歌了！,音乐,电音,普通视频,转载,"[传说之下, 电音, 音乐, 电音, UNDERTALE, underverse, nigh...",02:45,7681,558,33,349,24,NaN,22,否
1Ai4y1g72E,8月,不务正业的皮哥,无,2018,143,传奇！！世界DJ巨星David Guetta现场用Ableton编曲,音乐,电音,普通视频,转载,"[EDM, 电音, 音乐, 电音, 电音教程, David Guetta]",07:08,7670,272,26,264,30,NaN,26,否
1kh411973T,8月,Angerfeet,无,417,46,这个视频会彻底颠覆你对Medly的认知,音乐,电音,普通视频,自制,"[电音, 音乐, 电音, Hard Style, HARDSTYLE, Rawstyle, ...",03:33,7527,487,93,205,28,NaN,41,否
